In [1]:
import time
from pprint import pprint
from tqdm import tqdm
import openai

#avenieca sdk imports
from avenieca import Signal
from avenieca.producers import Event
from avenieca.api.model import *
from avenieca.api.eca import ECA
from avenieca.utils import hash

#local file imports
from config import *
import utils as util

load_dotenv()
data_path = os.getenv("DATA_PATH")
url = '%s/events_estore.csv' % data_path

def prettyprint(res, status=None):
    try:
        pprint(res.__dict__)
    except:
        print(res)
    if status:
        print(status)

In [2]:
import pandas as pd
data = pd.read_csv(url)

### Write directly to the ECA database

In [3]:
event = data['event'].values
item = data['item'].values

eca_server = os.getenv("ECA_SERVER")
eca_secret = os.getenv("ECA_SECRET")
openai.api_key = openai_key

config = Config(uri=eca_server, username=username, password=password)
eca = ECA(config=config)

for i in tqdm(range(0, len(event))):
    event_ess = util.create_ess_and_sequence(eca, event[i], eca_secret, event_twin_config, openai, True)
    item_ess = util.create_ess_and_sequence(eca, item[i], eca_secret, item_twin_config, openai, True)

    aggregate_in_twins = [
        event_ess,
        item_ess,
    ]
    aggregate_insert = ESSInsert(
        module_id=aggregate_config.module_id,
        state=[],
        valence=0.0,
    )
    aggregate_insert = util.create_aggregate_from_ess(aggregate_in_twins, aggregate_insert)
    util.search_insert_ess_and_sequence(eca, aggregate_insert)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95/95 [00:06<00:00, 14.42it/s]


### Recommendations

In [13]:
event = "addtocart"
input_hash = hash.encode(event, eca_secret)
emb_inp, status = eca.embedding.get_one_with_hash(module_id=event_twin_config.module_id, data=EmbeddingInputHash(
    hash=input_hash
))
prettyprint(emb_inp, status)

{'created_at': '2023-05-31T10:25:43.384438Z',
 'hash': 'XLOCS/nYnaSjdvLqFi9ZRdqklPT043K2RZ+TgPjSUkY=',
 'id': 2,
 'input': 'addtocart',
 'module_id': 'recsys_event',
 'updated_at': '2023-05-31T10:25:43.384438Z'}
200


In [14]:
res, status = eca.ess.get_one_with_embedding(module_id=event_twin_config.module_id, emb_input=emb_inp.id)
prettyprint(res.id)
event_ess: ESSResponse = res

2


In [6]:
item = "GOLF PARTY DECORATIONS Happy Birthday Giant Banner with Stickers"
item_ess = util.create_ess_and_sequence(eca, item, eca_secret, item_twin_config, openai, True)
item_ess.id

62

In [7]:
res, status = eca.ess.get_one_pretty(module_id=item_twin_config.module_id, db_id=62)
prettyprint(res)

{'aggregate_context': [],
 'aggregate_emb_inp': [],
 'aggregate_id': [],
 'aggregate_module_id': [],
 'aggregate_score': [],
 'aggregate_shape': [],
 'aggregate_valence': [],
 'avg_ess_score': 0,
 'avg_ess_valence': 0.0,
 'context': None,
 'created_at': '2023-05-31 12:06:03.581235 UTC',
 'embedding_input': None,
 'id': 62,
 'module_id': 'recsys_item',
 'score': 0,
 'state': None,
 'state_float': None,
 'state_str': 'GOLF PARTY DECORATIONS Happy Birthday Giant Banner with '
              'Stickers',
 'total_ess_score': 0,
 'updated_at': '2023-05-31 12:06:03.581235 UTC',
 'valence': 50.0}


In [15]:
aggregate_in_twins = [
    event_ess,
    item_ess,
]
aggregate_insert = ESSInsert(
    module_id=aggregate_config.module_id,
    state=[],
    valence=0.0,
)
aggregate_insert = util.create_aggregate_from_ess(aggregate_in_twins, aggregate_insert)
aggregate = util.search_insert_ess_and_sequence(eca, aggregate_insert)
aggregate.id

83

In [16]:
res, status = eca.ess.get_one_pretty(module_id=aggregate_config.module_id, db_id=83)
prettyprint(res)

{'aggregate_context': [None, None],
 'aggregate_emb_inp': [2, 62],
 'aggregate_id': [2, 62],
 'aggregate_module_id': ['recsys_event', 'recsys_item'],
 'aggregate_score': [0, 0],
 'aggregate_shape': [1536, 1536],
 'aggregate_valence': [50.0, 50.0],
 'avg_ess_score': 0,
 'avg_ess_valence': 50.0,
 'context': None,
 'created_at': '2023-05-31 12:10:06.113701 UTC',
 'embedding_input': None,
 'id': 83,
 'module_id': 'recsys_aggregate',
 'score': 0,
 'state': ['addtocart',
           'GOLF PARTY DECORATIONS Happy Birthday Giant Banner with Stickers'],
 'state_float': None,
 'state_str': None,
 'total_ess_score': 0,
 'updated_at': '2023-05-31 12:10:06.113701 UTC',
 'valence': 100.0}


In [18]:
nsr = NextStateRequest(
    module_id=aggregate_config.module_id,
    recall=3,
    range=2,
    n=5,
    status='n',
    current_state=83,
    previous_state=[60, 20]
)
res, status = eca.cortex.predictions(data=nsr)
prettyprint(res, status)

{'current_state': [{'aggregate_id': 83,
                    'ess_id': 2,
                    'module_id': 'recsys_event',
                    'state': 'addtocart'},
                   {'aggregate_id': 83,
                    'ess_id': 62,
                    'module_id': 'recsys_item',
                    'state': 'GOLF PARTY DECORATIONS Happy Birthday Giant '
                             'Banner with Stickers'}],
 'next_state': [{'list': [{'aggregate_id': 41,
                           'ess_id': 2,
                           'module_id': 'recsys_event',
                           'state': 'addtocart'},
                          {'aggregate_id': 41,
                           'ess_id': 34,
                           'module_id': 'recsys_item',
                           'state': 'Gyro Flyer XL Green'}]},
                {'list': [{'aggregate_id': 56,
                           'ess_id': 2,
                           'module_id': 'recsys_event',
                           'state': 'addt